In [1]:
import pandas as pd
import json
import glob

In [2]:
def get_spells_df():
    files = glob.glob('*.json')
    files.remove('_Template.json')

    result = list()
    for file_name in files:
        with open(file_name, 'r') as file:
            result.append(json.load(file))

    return pd.DataFrame(result).sort_values(by=['name', 'nivel']).reset_index(drop=True)

In [3]:
def fill_na_by_column():
    spells_df.elementos = spells_df.elementos.apply(lambda x: x if isinstance(x, list) else [])
    spells_df.attack_save.fillna('N/A', inplace=True)
    spells_df.dmg_effect.fillna('N/A', inplace=True)
    spells_df.dmg.fillna('N/A', inplace=True)
    spells_df.ritual.fillna(False, inplace=True)

def assert_column_not_null(column):
    if any(spells_df[column].isna()):
        raise ValueError(f"Column '{column}' shouldn't have null values!")

def assert_columns_not_null():
    not_null_columns = ['nome', 'name', 'nivel', 'escola', 'tempo_conjuracao', 'alcance_area', 
                        'componentes', 'mana', 'duracao', 'classes', 'tags', 'descricao', 'source']
    for column in not_null_columns:
        assert_column_not_null(column)

In [4]:
spells_df = get_spells_df()
assert_columns_not_null()
fill_na_by_column()
spells_df

,nome,name,nivel,escola,ritual,elementos,tempo_conjuracao,alcance_area,componentes,mana,duracao,attack_save,dmg_effect,dmg,classes,tags,descricao,source,adicional_mana,mana_adicional
0,Espiro Ácido,Acid Splash,0,elemental,False,[veneno],1 ação,18 metros,VS,50,instantânea,DEX Save,ácido,1d6,"[mago, ladino]",[dano],Você arremessa uma bolha de ácido. Escolha uma...,LDJ,NaN,NaN
1,Proteção contra Lâminas,Blade Ward,0,elemental,False,"[água, terra, ar, metal]",1 ação,pessoal,VS,100,1 rodada,N/A,N/A,N/A,"[mago, bardo]","[defesa, combate]",[Ar] Você estende sua mão e traça uma parede d...,LDJ,NaN,NaN
2,Lâmina Estrondosa,Booming Blade,0,elemental,False,"[relâmpago, ar, luz, sombras]",1 ação,pessoal,VSM (arma marcial corpo-a-corpo),100,1 rodada,corpo-a-corpo,elemental,1d8,"[ladino, guerreiro, paladino]","[dano, combate, controle, corpo-a-corpo, arma]",Você brande a arma utilizada na conjuração des...,Tasha,NaN,NaN
3,Toque Arrepiante,Chill Touch,0,necromancia,False,[],1 ação,36 metros,VS,100,1 rodada,N/A,necrótico,1d8,"[mago, bruxo, feiticeiro, guerreiro, ladino]","[dano, corpo-a-corpo]",Você cria uma mão esquelética fantasmagórica n...,LDJ,NaN,NaN
4,Controlar Chamas,Control Flames,0,elemental,False,"[fogo, sombras]",1 ação,18 metros,S,100,instantânea ou 1 hora,N/A,N/A,N/A,"[mago, feiticeiro, bruxo]","[utilidade, controle]",Você escolhe uma chama não mágica que você pos...,Xanathar,NaN,NaN
5,Criar Fogueira Instantânea,Create Bonfire,0,elemental,False,[fogo],1 ação,18 metros,VS,150,instantânea,DEX Save,fogo,1d8,"[guerreiro, paladino, ladino, clérigo]",[dano],Você cria uma fogueira no solo em um ponto que...,Xanathar,NaN,NaN
6,Criar Fogueira,Create Bonfire,0,elemental,False,[fogo],1 ação,18 metros,VS,100,"concentração, até 1 minuto",DEX Save,fogo,1d8,"[mago, feiticeiro, bruxo]","[dano, controle]",Você cria uma fogueira no solo em um ponto que...,Xanathar,NaN,NaN
7,Globos de Luz,Dancing Lights,0,elemental,False,"[luz, fogo]",1 ação,36 metros,VSM (um pouco de fósforo ou wychwood ou um ins...,50,"concentração, até 1 minuto",N/A,N/A,N/A,"[clérigo, paladino, bardo, feiticeiro, mago, l...",[utilidade],Você cria até quatro luzes do tamanho de tocha...,LDJ,NaN,NaN
8,Rajada Mística,Eldritch Blast,0,elemental,False,"[água, terra, ar, fogo, luz, sombras, relâmpago]",1 ação,36 metros,VS,150,instantânea,distância,elemental,1d10,"[clérigo, paladino, guerreiro, ladino, monge, ...","[dano, distância]",Um feixe elemental vai em direção a uma criatu...,LDJ,NaN,NaN
9,Raio de Fogo,Fire Bolt,0,elemental,False,"[fogo, sombras, luz]",1 ação,36 metros,VS,150,instantânea,distância,elemental,1d10,"[paladino, ladino, guerreiro, mago, feiticeiro...","[dano, combate]",Você arremessa um cisco elemental em uma criat...,LDJ,NaN,NaN


In [5]:
import re

def return_columns_not_matching_mask(df, column, mask):
    return df[['nome', 'name', column]][~mask]

def get_mask_from_list(df, column, list_of_possible_values):
    unique_values = set(df[column].sum())
    wrong_values = list(filter(lambda x: x not in list_of_possible_values, unique_values))
    mask = df[column].apply(lambda a_list: any([True if element in wrong_values else False for element in a_list]))
    mask = ~mask
    return mask

def assert_column_using_mask(df, column, mask):
    if not mask.all():
        raise ValueError(f"Column '{column}' have wrong values:\n{return_columns_not_matching_mask(df, column, mask)}")

def assert_value_formats(df):
        escolas = ['elemental', 'necromancia', 'psíquica', 'ilusionista', 'invocação', 'espiritual', 'atrativa', 'musical', 'pura']
        mask = df.escola.str.islower()
        mask = mask & (df.escola.isin(escolas))
        assert_column_using_mask(df, 'escola', mask)

        elementos = ['ar', 'fogo', 'luz', 'metal', 'relâmpago', 'sombras', 'terra', 'veneno', 'água']
        mask = get_mask_from_list(df, 'elementos', elementos)
        assert_column_using_mask(df, 'elementos', mask)

        tempo_conjuracao_regex = re.compile(r'\d+\ (ação|ação bonus|minuto|ações|minutos)')
        mask = df.tempo_conjuracao.str.fullmatch(tempo_conjuracao_regex)
        assert_column_using_mask(df, 'tempo_conjucarao', mask)

        alcance_regex = re.compile(r'(pessoal|toque|\d+(,\d+)?\ metros?)')
        mask = df.alcance_area.str.fullmatch(alcance_regex)
        assert_column_using_mask(df, 'alcance_area', mask)

        componentes_regex = re.compile(r'V?S?M?(\ \(.+\))?')
        mask = df.componentes.str.fullmatch(componentes_regex)
        assert_column_using_mask(df, 'componentes', mask)

        mask = df.duracao.str.islower()
        assert_column_using_mask(df, 'duracao', mask)

        attack_save_regex = re.compile(r'(N/A|corpo-a-corpo|distância|(STR|DEX|CON|INT|WIS|CAR)\ (Test|Save))')
        mask = df.attack_save.str.fullmatch(attack_save_regex)
        assert_column_using_mask(df, 'attack_save', mask)

        tags = ['buff', 'debuff', 'controle', 'combate', 'utilidade', 'dano', 'defesa', 
                'cura', 'distância', 'corpo-a-corpo', 'toque', 'arma', 'comunicação']
        mask = get_mask_from_list(df, 'tags', tags)
        assert_column_using_mask(df, 'tags', mask)

        classes = ['bardo', 'monge', 'clérigo', 'paladino', 'ladino', 'guerreiro', 'mago', 'bruxo', 'feiticeiro']
        mask = get_mask_from_list(df, 'classes', classes)
        assert_column_using_mask(df, 'classes', mask)

        source_regex = re.compile(r'(LDJ|Xanathar|Tasha|Wildemount)')
        mask = df.source.str.fullmatch(source_regex)
        assert_column_using_mask(df, 'source', mask)

        
try:
    assert_value_formats(spells_df)
except ValueError as e:
    print('Wrong values.')
    print(e)

In [6]:
spells_df.nivel.unique()

array([0])

In [7]:
spells_df.escola.unique()

array(['elemental', 'necromancia', 'psíquica', 'invocação', 'ilusionista',
       'musical'], dtype=object)

In [8]:
set(spells_df.elementos.sum())

{'ar',
 'fogo',
 'luz',
 'metal',
 'relâmpago',
 'sombras',
 'terra',
 'veneno',
 'água'}

In [9]:
spells_df.tempo_conjuracao.unique()

array(['1 ação', '1 ação bonus', '1 minuto'], dtype=object)

In [10]:
spells_df.alcance_area.unique()

array(['18 metros', 'pessoal', '36 metros', 'toque', '9 metros',
       '4,5 metros', '3 metros', '1,5 metros'], dtype=object)

In [11]:
spells_df.componentes.unique()

array(['VS', 'VSM (arma marcial corpo-a-corpo)', 'S',
       'VSM (um pouco de fósforo ou wychwood ou um inseto luminoso)',
       'SM (uma pequena quantidade de maquiagem)',
       'SM (uma arma marcial corpo-a-corpo)', 'VSM (uma pulga viva)',
       'VSM (um vaga-lume ou musgo fosforescente)',
       'VSM (um pouco de pó de prata)', 'VSM (dois ímas)',
       'VSM (um pedaço curto de fio de cobre)', 'V',
       'VM (um pouco de lã)', 'VSM (um manto em miniatura)',
       'VSM (um instrumento musical)', 'SM (um instrumento musical)',
       'VSM (um instrumento musical ou canto)',
       'VSM (um instrumento musical ou a voz)'], dtype=object)

In [12]:
sorted(spells_df.mana.unique())

[50, 75, 100, 150, 200]

In [13]:
spells_df.duracao.unique()

array(['instantânea', '1 rodada', 'instantânea ou 1 hora',
       'concentração, até 1 minuto', 'concentração, até 1 hora',
       '1 minuto', 'instantâneo ou 1 hora', 'até 1 hora', '10 minutos',
       'até 1 minuto'], dtype=object)

In [14]:
spells_df.attack_save.unique()

array(['DEX Save', 'N/A', 'corpo-a-corpo', 'distância', 'CON Save',
       'STR Save', 'INT Save', 'INT Test', 'WIS Save'], dtype=object)

In [15]:
spells_df.dmg_effect.unique()

array(['ácido', 'N/A', 'elemental', 'necrótico', 'fogo', 'veneno',
       'concussão', 'psíquico', 'luz', 'relâmpago', 'energia'],
      dtype=object)

In [16]:
spells_df.dmg.unique()

array(['1d6', 'N/A', '1d8', '1d10', 'mod', '1d6 + mod', '1d12', '1d4',
       '1d8 ou 1d12'], dtype=object)

In [17]:
spells_df.source.unique()

array(['LDJ', 'Tasha', 'Xanathar', 'Wildemount'], dtype=object)

In [18]:
spells_df.adicional_mana.unique()

array([nan, '50 por turno'], dtype=object)

In [19]:
spells_df[spells_df['name'] == 'Guidance']

,nome,name,nivel,escola,ritual,elementos,tempo_conjuracao,alcance_area,componentes,mana,duracao,attack_save,dmg_effect,dmg,classes,tags,descricao,source,adicional_mana,mana_adicional
13,Orientação,Guidance,0,elemental,False,"[luz, água]",1 ação,toque,VS,75,"concentração, até 1 minuto",N/A,N/A,N/A,"[clérigo, paladino, bardo, bruxo, feiticeiro, ...",[buff],Você toca uma criatura voluntária e uma aura e...,LDJ,50 por turno,NaN


In [20]:
print(spells_df[spells_df['name'] == 'Guidance']['descricao'].iloc[0])

Você toca uma criatura voluntária e uma aura elemental envolve ela, desaparecendo após alguns segundos. Uma vez, antes da magia acabar, o alvo pode rolar 1d4 e adicionar o número rolado a um teste de habilidade a escolha dele. Ele pode rolar o dado antes ou depois de realizar o teste de habilidade. Após isso, a magia termina. Você gasta 50 de mana por turno até o final da magia.


In [21]:
print('Você toca uma criatura voluntária. Uma vez, antes da magia acabar, o alvo pode rolar um d4 e adicionar o número rolado a um teste de habilidade a escolha dele.  Ele pode rolar o dado antes ou depois de realizar o teste de habilidade. Após isso, a magia termina.')

Você toca uma criatura voluntária. Uma vez, antes da magia acabar, o alvo pode rolar um d4 e adicionar o número rolado a um teste de habilidade a escolha dele.  Ele pode rolar o dado antes ou depois de realizar o teste de habilidade. Após isso, a magia termina.


In [22]:
spells_df.mana.value_counts()

150    15
100    14
50     11
75      2
200     2
Name: mana, dtype: int64